## PyCrown tree delnieation at SPR



This notebook uses the pycrown module, modified to work with LAZ lidar point cloud data, with units in feet, if needed, to delineate tree crowns. Sample site at Cal Poly Swanton Pacific Ranch

Notes: 
- ran conda update pandas
- Removed Pandas from requirements.txt
- Updated Geopandas to 0.10.2 in requirements.txt
- Update requirements to newest, applicable versions

- 1ft gap between adjacent tiles, rounding error of 'AOI', that must be snapped to dem.

Rwhite 
Feb 2022

#### Installing PyCrown and dependencies

In [1]:
import os
import pandas as pd
#Creating a new folder to download pycrown
#os.chdir('/arcgis/home')
#os.mkdir('pycrown4')
os.chdir('/arcgis/home/pycrown4/pycrown')


In [ ]:
#Run once, Lets clone the modified pycrown repository from github
#!git clone https://github.com/russ-white/pycrown

In [2]:
#Installing the packages from the PyCrown requirements.txt
!pip install -r requirements.txt
#Installing Laspy with LasZip
!pip install laspy[laszip]
#Run the PyCrown Setup
!python setup.py install


     |████████████████████████████████| 31.2 MB 29.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 72.0 MB/s eta 0:00:01
     |████████████████████████████████| 598 kB 60.5 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.3 MB 44.3 MB/s eta 0:00:01
     |████████████████████████████████| 16.7 MB 60.0 MB/s eta 0:00:01
  Created wheel for laspy: filename=laspy-2.1.2-py3-none-any.whl size=52804 sha256=0ef026f64879335ffc4a2d837343db9a451e3ac676e9abcf78872c39c8c8ef8e
  Stored in directory: /home/arcgis/.cache/pip/wheels/e6/f9/4c/751d63180b7ad21ef3dfa98e249d76e0f95637f15d6268dadb
Successfully built laspy
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.2
    Uninstalling scipy-1.5.2:
      Successfully uninstalled scipy-1.5.2
  Attempting uninstall: Fiona
    Found existing installation: Fiona 1.7.12
    Uninstalling

In [3]:
conda update pandas

Solving environment: | 
  - defaults/linux-64::cryptography-3.4.8-py37hd23ed53_0, defaults/linux-64::pyjwt-2.1.0-py37h06a4308_0, defaults/noarch::oauthlib-3.2.0-pyhd3eb1b0_0, defaults/noarch::pyopenssl-20.0.1-pyhd3eb1b0_1
  - defaults/linux-64::cryptography-37.0.1-py37h9ce1e76_0, defaults/linux-64::pyjwt-1.7.1-py37_0, defaults/noarch::oauthlib-3.1.0-py_0, defaults/noarch::pyopenssl-22.0.0-pyhd3eb1b0done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.12.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |            1_gnu          22 KB  defaults
    absl-py-0.15.0             |     pyhd3eb1b0_0         103 KB  defaults
    affine-2.3.0    

In [2]:
#import geopandas
#import pygeos
#geopandas.options.use_pygeos = True

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.7.1-CAPI-1.11.1 0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


#### Get LiDAR Index and Download LAZ data for SPR Sample Site 1

In [4]:
os.chdir('/arcgis/home/pycrown4/pycrown')
# Connecting to ArcGIS Online
from arcgis import GIS
from arcgis.features import SpatialDataFrame
gis = GIS("https://calpoly.maps.arcgis.com/")


NameError: name 'Dataset' is not defined

In [ ]:
import sys
from datetime import datetime
from pycrown import PyCrown
import laspy
import numpy as np
#figure out how to get Cython - 'crown_dalponte_cython'

# Slite Tiles and save them to the files

In [ ]:
os.chdir("/arcgis/home/pycrown4/pycrown/raster_test")
os.mkdir("tiles_DTM")
os.mkdir("tiles_DSM")
os.mkdir("tiles_CHM")

In [9]:
import arcpy

arcpy.management.SplitRaster("/arcgis/home/pycrown4/pycrown/site_data/spr_site_1/raster/SCC_2020_CHM.tif", "/arcgis/home/pycrown4/pycrown/raster_test/tiles_CHM", "CHM_", "SIZE_OF_TILE", "TIFF", "CUBIC", "1 1", "1000 1000", 100, "PIXELS", None, None, None, "NONE", "DEFAULT", '')


<Result '/arcgis/home/pycrown4/pycrown/raster_test/tiles_CHM'>

In [10]:
import arcpy

arcpy.management.SplitRaster("/arcgis/home/pycrown4/pycrown/site_data/spr_site_1/raster/SCC_2020_DEM.tif", "/arcgis/home/pycrown4/pycrown/raster_test/tiles_DTM", "DTM_", "SIZE_OF_TILE", "TIFF", "CUBIC", "1 1", "1000 1000", 100, "PIXELS", None, None, None, "NONE", "DEFAULT", '')


<Result '/arcgis/home/pycrown4/pycrown/raster_test/tiles_DTM'>

In [11]:
import arcpy

arcpy.management.SplitRaster("/arcgis/home/pycrown4/pycrown/site_data/spr_site_1/raster/SCC_2020_DSM.tif","/arcgis/home/pycrown4/pycrown/raster_test/tiles_DSM", "DSM_", "SIZE_OF_TILE", "TIFF", "CUBIC", "1 1", "1000 1000", 100, "PIXELS", None, None, None, "NONE", "DEFAULT", '')


<Result '/arcgis/home/pycrown4/pycrown/raster_test/tiles_DSM'>

import os
import glob

files = '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2020/'

for file in os.listdir(files):
  
    try:
        os.remove(f)
    except OSError as e:
        print("Error: %s : %s" % (f, e.strerror))

In [44]:
os.chdir("/arcgis/home/pycrown4/pycrown")
os.mkdir("Tests_SPRING_2022")

FileExistsError: [Errno 17] File exists: 'Tests_SPRING_2022'

In [11]:
from datetime import datetime
#os.chdir("/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022")
datetime = datetime.now().strftime("%d-%m-%Y_%H:%M:%S")
#os.mkdir("Resalts_Test_" + datetime  )

In [13]:
trees_merge = pd.DataFrame(columns=[
    'top_height', 'top_elevation',
    'top_cor_height', 'top_cor_elevation',
    'crown_poly_raster', 'crown_poly_smooth',
    'top_cor', 'top', 'tt_corrected'
])

trees_merge = trees_merge.astype(dtype={
    'top_height': 'float',
    'top_elevation': 'float',
    'top_cor_height': 'float',
    'top_cor_elevation': 'float',
    'crown_poly_raster': 'object',
    'crown_poly_smooth': 'object',
    'top_cor': 'object',
    'top': 'object',
    'tt_corrected': 'int'
})

In [15]:
os.chdir("/arcgis/home/pycrown4/pycrown")

import fiona
from fiona.crs import from_epsg
import shapely
from shapely.geometry import mapping, Point, Polygon
import time
out = '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_' +  datetime 
CHM_PATH = "/arcgis/home/pycrown4/pycrown/raster_test/tiles_CHM"

DTM_PATH ="/arcgis/home/pycrown4/pycrown/raster_test/tiles_DTM" 

DSM_PATH ="/arcgis/home/pycrown4/pycrown/raster_test/tiles_DSM"

F_LAS = None

start = time.time()

for file in os.listdir(CHM_PATH):
    if file.endswith(".TIF"):
        
        Ending = file.partition("_")[2]
        Number = Ending.partition(".")[0]

        print(Number)
        print(Ending)
        CHM_file = CHM_PATH + "/CHM_" + Ending 
        DTM_file = DTM_PATH + "/DTM_" + Ending
        DSM_file = DSM_PATH + "/DSM_" + Ending
        
        print(CHM_file)
        print(DTM_file)
        print(DSM_file)
        try:
            PC = PyCrown(CHM_file, DTM_file, DSM_file, F_LAS ,outpath = out)
            PC.filter_chm(3,ws_in_pixels=True, circular=False)
        
            PC.tree_detection(PC.chm, ws=3, ws_in_pixels=True, hmin=6)
            print(f"Number of trees detected: {len(PC.trees)}")  

        
            PC.crown_delineation(algorithm='dalponteCIRC_numba', th_tree=6, th_seed=0.45, th_crown=0.55, max_crown=30)
            
            PC.clip_trees_to_bbox(inbuf=150)
            
            PC.get_tree_height_elevation(loc='top')
            #PC.screen_small_trees(hmin=6, loc='top')

            PC.crowns_to_polys_raster()

            trees_merge = pd.concat([trees_merge, PC.trees])
           
            #RIGHT TO A text file with date that we did
        except:
    
            pass
    
        
           
endtile = time.time()
    
print(endtile - start)   




4
4.TIF
/arcgis/home/pycrown4/pycrown/raster_test/tiles_CHM/CHM_4.TIF
/arcgis/home/pycrown4/pycrown/raster_test/tiles_DTM/DTM_4.TIF
/arcgis/home/pycrown4/pycrown/raster_test/tiles_DSM/DSM_4.TIF
Number of trees detected: 78
Tree crowns delineation: 0.002s
5
5.TIF
/arcgis/home/pycrown4/pycrown/raster_test/tiles_CHM/CHM_5.TIF
/arcgis/home/pycrown4/pycrown/raster_test/tiles_DTM/DTM_5.TIF
/arcgis/home/pycrown4/pycrown/raster_test/tiles_DSM/DSM_5.TIF
Number of trees detected: 213
Tree crowns delineation: 0.001s
6
6.TIF
/arcgis/home/pycrown4/pycrown/raster_test/tiles_CHM/CHM_6.TIF
/arcgis/home/pycrown4/pycrown/raster_test/tiles_DTM/DTM_6.TIF
/arcgis/home/pycrown4/pycrown/raster_test/tiles_DSM/DSM_6.TIF
Number of trees detected: 960
Tree crowns delineation: 0.004s
7
7.TIF
/arcgis/home/pycrown4/pycrown/raster_test/tiles_CHM/CHM_7.TIF
/arcgis/home/pycrown4/pycrown/raster_test/tiles_DTM/DTM_7.TIF
/arcgis/home/pycrown4/pycrown/raster_test/tiles_DSM/DSM_7.TIF
Number of trees detected: 2989
Tree cro

In [6]:
from arcgis.gis import GIS
gis=GIS("https://arcgis.com", "joogden_CalPoly")

NameError: name 'Dataset' is not defined

In [19]:
import geopandas
gdf_crowns = geopandas.GeoDataFrame(trees_merge, crs={'init': 'epsg:6420'}, geometry='crown_poly_raster')
gdf_tops = geopandas.GeoDataFrame(trees_merge, crs={'init': 'epsg:6420'}, geometry='top')


In [7]:
from arcgis.features import SpatialDataFrame, Feature, FeatureSet
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import pandas as pd
from arcgis import geometry


NameError: name 'Dataset' is not defined

In [ ]:
#Create an ESRI Spatially Enabled Data Frame from GeoDataFrame
sedf_tops=GeoAccessor.from_geodataframe(gdf_tops, column_name="geometry")
sedf_crowns=GeoAccessor.from_geodataframe(gdf_crowns, column_name="geometry") 

In [ ]:
#Dropping all the Null and extra fields
sedf_tops = sedf_tops.drop(['top_cor_height', 'top_cor_elevation', 'crown_poly_smooth', 'tt_corrected','top_cor', 'crown_poly_raster'],1)
sedf_crowns = sedf_crowns.drop(['top_cor_height', 'top_cor_elevation', 'crown_poly_smooth', 'tt_corrected','top_cor', 'top'],1)

sedf_crowns

In [1]:
#Publish as a layer in ArcGIS Online
#Check for way to overwrite
lyr = sedf_tops.spatial.to_featurelayer('tree_tops_spr' + datetime )
#Publish as a layer in ArcGIS Online
#Check for way to overwrite
#The crowns layer takes a bit longer than the point layer
lyr = sedf_crowns.spatial.to_featurelayer('tree_crowns_spr' + datetime )

NameError: name 'sedf_tops' is not defined

In [ ]:
#next steps: santcruse county has bulding footprints open street map 


In [ ]:
# Item Added From Toolbar
# Title: OpenStreetMap Buildings for North America | Type: Feature Service | Owner: smoore_osm
buildings = gis.content.get("8573b205d20d4be38b6e11eb05dd151b")
buildings

In [ ]:
#find the aria we are looking at, find those buildings, remove tose "trees" 

#look at api documentation



In [ ]:
#old stuff unnder me

In [18]:
os.chdir("/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022")
os.mkdir("Resalts_Test_Merge_" + datetime )

In [19]:
#Merge tree crown result shapefiles to single shapefile
import arcpy
infile = '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_' +  datetime  
out = '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_Merge_' + datetime  +"/"
matches = []

for files in os.listdir(infile):
    #identify the shapefiles
    if files.endswith(".shp"):
        #get the ones that are 'crowns'
        result_type = files.split("_")[1]
        #print(result_type)
        if result_type == "crown":
            #create a list of crown shapefiles to merge
            match = (os.path.join(infile, files))
            matches.append (match)
                   
print(matches)
#Merge into one shapefile, using arcpy Merge
arcpy.Merge_management(matches, os.path.join(out,"tree_crown_merge.shp"))

['/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_06-05-2022_20:56:24/tree_crown_4.shp', '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_06-05-2022_20:56:24/tree_crown_5.shp', '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_06-05-2022_20:56:24/tree_crown_6.shp', '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_06-05-2022_20:56:24/tree_crown_7.shp', '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_06-05-2022_20:56:24/tree_crown_8.shp', '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_06-05-2022_20:56:24/tree_crown_9.shp', '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_06-05-2022_20:56:24/tree_crown_10.shp', '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_06-05-2022_20:56:24/tree_crown_11.shp', '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_06-05-2022_20:56:24/tree_crown_12.shp', '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_06-05-2022_20:56:24/tree_

<Result '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_Merge_06-05-2022_20:56:24/tree_crown_merge.shp'>

In [ ]:
dataset = '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_Merge_06-05-2022_20:56:24/tree_crown_merge.cpg'

In [ ]:
#Now, repeating for the tree tops point results
import arcpy
infile = '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_' +  datetime 
out = '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_Merge_' + datetime  + "/"
matches = []

for files in os.listdir(infile):
    #print(files)
    if files.endswith(".shp"):
        result_type = files.split("_")[1]
        #print(result_type)
        if result_type == "tops":
            match = (os.path.join(infile, files))
            matches.append (match)
                   
print(matches)
arcpy.Merge_management(matches, os.path.join(out,"tree_tops_merge.shp"))

['/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_06-05-2022_20:56:24/tree_tops_4.shp', '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_06-05-2022_20:56:24/tree_tops_5.shp', '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_06-05-2022_20:56:24/tree_tops_6.shp', '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_06-05-2022_20:56:24/tree_tops_7.shp', '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_06-05-2022_20:56:24/tree_tops_8.shp', '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_06-05-2022_20:56:24/tree_tops_9.shp', '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_06-05-2022_20:56:24/tree_tops_10.shp', '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_06-05-2022_20:56:24/tree_tops_11.shp', '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_06-05-2022_20:56:24/tree_tops_12.shp', '/arcgis/home/pycrown4/pycrown/Tests_SPRING_2022/Resalts_Test_06-05-2022_20:56:24/tree_tops_13.s

In [ ]:
os.chdir("/arcgis/home/pycrown4/pycrown")

import fiona
from fiona.crs import from_epsg
import shapely
from shapely.geometry import mapping, Point, Polygon
import time
out = '/arcgis/home/pycrown4/pycrown/Resalts_Test/'

CHM_PATH = "/arcgis/home/pycrown4/pycrown/raster_test/tiles_CHM"

DTM_PATH ="/arcgis/home/pycrown4/pycrown/raster_test/tiles_DEM" 

DSM_PATH ="/arcgis/home/pycrown4/pycrown/raster_test/tiles_DSM"

F_LAS = None

start = time.time()

for CHM_file in os.listdir(CHM_PATH):
    if filename.endswith(".TIF"):
      Ending = filename.partition("_")[2]
      DEM_file = DTM_PATH + "\DTM_" + Ending
      DSM_file = DSM_PATH + "\DSM_" + Ending
      print(CHM_file)
      print(DEM_file)
      print(DSM_file)
      
 

PC = PyCrown(F_CHM, F_DTM, F_DSM, F_LAS ,outpath = out)

PC.filter_chm(3,ws_in_pixels=True, circular=False)
        
PC.tree_detection(PC.chm, ws=3, ws_in_pixels=True, hmin=6)

print(f"Number of trees detected: {len(PC.trees)}")  

        
PC.crown_delineation(algorithm='dalponteCIRC_numba', th_tree=6, th_seed=0.45, th_crown=0.55, max_crown=30)

        
#PC.clip_trees_to_bbox(bbox=ext)
#Lets skip the 'correct tree top height' step, since it takes forever
#PC.correct_tree_tops()
PC.get_tree_height_elevation(loc='top')

#and here
#PC.get_tree_height_elevation(loc='top_cor')
PC.screen_small_trees(hmin=6, loc='top')

PC.crowns_to_polys_raster()





#TODO TIF files for each file in in raster_test if .TIF prosses/ if CHM and a TIF. keep Number for nameing

In [14]:
#TODO add tile number onto shape file to keep track of resalts
out_trees = out+'tree_tops.shp'
tree_schema = {'geometry': '3D Point',
        'properties': {'DN': 'int', 'TH': 'float'}}

with fiona.collection(
    str(out_trees), 'w', 'ESRI Shapefile', tree_schema
    ) as output:
        for tidx in range(len(PC.trees['top'])):
            feat = {}
            tree_top = PC.trees['top'].iloc[tidx]
            feat['geometry'] = mapping(PC.trees['top'].iloc[tidx])
            feat['properties'] = {'DN': tidx,'TH': float(PC.trees.iloc[tidx].top_height)
            }
            output.write(feat)

#skipping this
#PC.export_tree_locations(loc='top_cor')

outfile = out+'tree_crown.shp'
schema = {'geometry': 'Polygon',
      'properties': {'DN': 'int', 'TTH': 'float'}}
with fiona.collection(
    str(outfile), 'w', 'ESRI Shapefile', schema
    ) as output:
        for tidx in range(len(PC.trees['crown_poly_raster'])):
            feat = {}
            tree = PC.trees['crown_poly_raster'].iloc[tidx]
            feat['geometry'] = mapping(PC.trees['crown_poly_raster'].iloc[tidx])
            feat['properties'] = {'DN': tidx,'TTH': float(PC.trees.iloc[tidx].top_height)
            }
            output.write(feat)

In [6]:
PC.trees

,top_height,top_elevation,top_cor_height,top_cor_elevation,crown_poly_raster,crown_poly_smooth,top_cor,top,tt_corrected
0,92.033936,4.280961e+02,NaN,NaN,"POLYGON ((6071015.999999997 1840016.99999999, ...",NaN,POINT (6071017.499999997 1840015.49999999),POINT (6071017.499999997 1840015.49999999),NaN
1,88.488922,4.184019e+02,NaN,NaN,"POLYGON ((6071207.999999997 1840016.99999999, ...",NaN,POINT (6071029.499999997 1840015.49999999),POINT (6071029.499999997 1840015.49999999),NaN
2,142.692200,3.952083e+02,NaN,NaN,"POLYGON ((6071492.999999997 1840016.99999999, ...",NaN,POINT (6071062.499999997 1840015.49999999),POINT (6071062.499999997 1840015.49999999),NaN
3,59.506775,4.368494e+02,NaN,NaN,"POLYGON ((6071795.999999997 1840016.99999999, ...",NaN,POINT (6071209.499999997 1840015.49999999),POINT (6071209.499999997 1840015.49999999),NaN
4,71.834106,4.297487e+02,NaN,NaN,"POLYGON ((6071879.999999997 1840016.99999999, ...",NaN,POINT (6071257.499999997 1840015.49999999),POINT (6071257.499999997 1840015.49999999),NaN
...,...,...,...,...,...,...,...,...,...
7940,68.599426,-3.402823e+38,NaN,NaN,"POLYGON ((6073508.999999997 1837028.99999999, ...",NaN,POINT (6073882.499999997 1837018.49999999),POINT (6073882.499999997 1837018.49999999),NaN
7941,38.917664,-3.402823e+38,NaN,NaN,"POLYGON ((6073685.999999997 1837055.99999999, ...",NaN,POINT (6073915.499999997 1837018.49999999),POINT (6073915.499999997 1837018.49999999),NaN
7942,72.458435,-3.402823e+38,NaN,NaN,"POLYGON ((6073805.999999997 1837031.99999999, ...",NaN,POINT (6073934.999999997 1837018.49999999),POINT (6073934.999999997 1837018.49999999),NaN
7943,86.471375,-3.402823e+38,NaN,NaN,"POLYGON ((6073853.999999997 1837046.99999999, ...",NaN,POINT (6073973.999999997 1837018.49999999),POINT (6073973.999999997 1837018.49999999),NaN


In [4]:
#LAZ tile Index from the USGS 
tile_layer = gis.content.get("5cecb6b6d91749b38510e7e9efdb9f6e")

#Get the tile_layer 'endpoint' url
tile_layer_endpoint = tile_layer.layers[0]
tile_layer_endpoint

#view the basic layer info
tile_layer

<Item title:"USGS_LiDAR_Tile_Index_CA_SantaCruzCounty_2020_A20" type:Feature Layer Collection owner:rwhite>

In [5]:
#Show the layer attribute table
index_table = tile_layer_endpoint.query(where='1=1').sdf
index_table.head(5)

,OBJECTID,tile_id,project,project_id,workunit_id,URL,Shape__Area,Shape__Length,SHAPE
0,1,040872,CA_SantaCruzCounty_2020_A20,196602,196600,https://rockyweb.usgs.gov/vdelivery/Datasets/S...,915111.546875,3826.467271,"{""rings"": [[[-13613191.0702426, 4456363.445861..."
1,2,040870,CA_SantaCruzCounty_2020_A20,196602,196600,https://rockyweb.usgs.gov/vdelivery/Datasets/S...,914944.667969,3826.118360,"{""rings"": [[[-13613172.8222633, 4455405.043563..."
2,3,040867,CA_SantaCruzCounty_2020_A20,196602,196600,https://rockyweb.usgs.gov/vdelivery/Datasets/S...,914777.773438,3825.769388,"{""rings"": [[[-13613154.5775039, 4454446.728404..."
3,4,040865,CA_SantaCruzCounty_2020_A20,196602,196600,https://rockyweb.usgs.gov/vdelivery/Datasets/S...,914610.808594,3825.420230,"{""rings"": [[[-13613136.3359564, 4453488.499992..."
4,5,040862,CA_SantaCruzCounty_2020_A20,196602,196600,https://rockyweb.usgs.gov/vdelivery/Datasets/S...,914444.121094,3825.071636,"{""rings"": [[[-13613118.0980026, 4452530.358680..."


In [6]:
# Introducing a new layer, our 'Area of Interest' or aoi, polygon.
# This is a layer we can draw to choose the area for analysis
# For now, we'll just focus on the same area
aoi = gis.content.get("a137757a33ca41768f7573fb8591821a")
aoi_endpoint = aoi.layers[0]
aoi

<Item title:"Forest_AOI_View" type:Feature Layer Collection owner:rwhite_CalPoly>

In [7]:
#Select one specific polygon to run, eg item number 5
aoi_selected = aoi_endpoint.query(where="ObjectID = 14")
aoi_extent = aoi_selected.features[0].geometry
aoi_extent

{'rings': [[[-13433580.2839, 4206365.8242],
   [-13431043.4403, 4206433.3888],
   [-13430985.5616, 4204231.0337],
   [-13433521.9086, 4204163.4956],
   [-13433580.2839, 4206365.8242]]]}

In [7]:
#Testing, This gets the Min and Max extents of the LAZ tiles, that overlap the red box
aoi_tiles = tile_layer_endpoint.query(geometry=aoi_extent, geometryType='esriGeometryPolygon', spatialRel='esriSpatialRelIntersects', returnExceededLimitFeatures='true', returnIdsOnly='false', inSR = '3857',outSR='6420', as_df='true')
selected = pd.Series.tolist(aoi_tiles['tile_id'])
print(selected)

#print('Min X: '+str(aoi_tiles['SHAPE'][0]['rings'][0][0][0]))
#print('Max X: '+str(aoi_tiles['SHAPE'][0]['rings'][0][2][0]))
#print('Min Y: '+str(aoi_tiles['SHAPE'][0]['rings'][0][0][1]))
#print('Max Y: '+str(aoi_tiles['SHAPE'][0]['rings'][0][1][1]))

#testing out printing the extents, all together now
#str(aoi_tiles['SHAPE'][0]['rings'][0][0][0])+','+str(aoi_tiles['SHAPE'][0]['rings'][0][2][0])+','+str(aoi_tiles['SHAPE'][0]['rings'][0][0][1])+','+str(aoi_tiles['SHAPE'][0]['rings'][0][1][1])
print(str(len(selected))+' tiles')


['060850', '060847', '057850', '057847']
4 tiles


In [8]:
aoi_selected_ft = aoi_endpoint.query(where="ObjectID = 14", outSR='3498')
#This gets the extent of the red box
aoi_extent_ft = aoi_selected_ft.features[0].geometry
aoi_xmin = aoi_extent_ft['rings'][0][0][0]
aoi_xmax = aoi_extent_ft['rings'][0][2][0]
aoi_ymin = aoi_extent_ft['rings'][0][2][1]
aoi_ymax = aoi_extent_ft['rings'][0][0][1]
aoi_bbox = (aoi_xmin, aoi_xmax, aoi_ymin, aoi_ymax)
print('Alternatively, This could be used to process the whole extent in one chunk')
aoi_bbox

Alternatively, This could be used to process the whole extent in one chunk


(5763402.2782367, 5770203.72295616, 2304625.17225101, 2310503.98621985)

In [9]:
#Adapting the loop to export out the Extents for each LAZ tile, as a tuple named ext, to be used in PC clip 
i=0
tile_ext = [None] * len(selected)
for i in range(len(selected)):
    #tile_ext[i] = str(aoi_tiles['SHAPE'][i]['rings'][0][0][0])+','+str(aoi_tiles['SHAPE'][i]['rings'][0][2][0])+','+str(aoi_tiles['SHAPE'][i]['rings'][0][0][1])+','+str(aoi_tiles['SHAPE'][i]['rings'][0][1][1])
    xmin = aoi_tiles['SHAPE'][i]['rings'][0][2][0]
    xmax = aoi_tiles['SHAPE'][i]['rings'][0][0][0]
    ymin = aoi_tiles['SHAPE'][i]['rings'][0][1][1]
    ymax = aoi_tiles['SHAPE'][i]['rings'][0][0][1]
    
    ext = (xmin,xmax,ymin,ymax)
    print(ext)
    


(6062499.98992594, 6059999.9995191, 1852499.9895437, 1849999.99946496)
(6062499.98959787, 6059999.99951911, 1849999.98962246, 1847499.9998718)
(6059999.9896766, 6057499.99959785, 1852499.98954371, 1849999.99979305)
(6059999.98967661, 6057499.99959785, 1849999.98962246, 1847499.99987181)


In [ ]:
#Get file download URLs from the table, as a list
download_list = pd.Series.tolist(aoi_tiles['URL'])
download_list

In [15]:
# If needed, create a data folder for this site, within pycrown folder
#os.mkdir('/arcgis/home/pycrown4/pycrown/site_data/spr_site_1/laz')
#if needed, a las folder
#os.mkdir('/arcgis/home/pycrown4/pycrown/site_data/spr_site_1/las')
#and, a results folder
#os.mkdir('/arcgis/home/pycrown4/pycrown/site_data/spr_site_1/results')

In [17]:
#Download the selected laz files from the USGS link to site_data/laz
import os
import urllib
import urllib.request
download_dir = '/arcgis/home/pycrown4/pycrown/site_data/spr_site_1/laz'


# For every line in the file
for url in download_list:
    # Split on the rightmost _ and take everything on the right side of that
    name = url.rsplit('_', 1)[-1]
    #name = name.rstrip('\n')
    # Combine the name and the downloads directory to get the local filename
    filename = os.path.join(download_dir, name)

    # Download the file if it does not exist
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(url, filename)

#### Running PyCrown for SPR Sample Site 1

In [9]:
#make a new results folder
#os.mkdir('/arcgis/home/pycrown4/pycrown/site_data/campus_site/raster/')
out = '/arcgis/home/pycrown4/pycrown/site_data/campus_site/results_2/'

In [10]:
os.chdir('/arcgis/home/pycrown4/pycrown')

In [11]:
import sys
from datetime import datetime
from pycrown import PyCrown
import laspy
import numpy as np
#figure out how to get Cython - 'crown_dalponte_cython'

##### Setup input files

In [12]:
F_CHM = '/arcgis/home/pycrown4/pycrown/site_data/campus_site/raster/campus_chm.tif'
F_DTM = '/arcgis/home/pycrown4/pycrown/site_data/campus_site/raster/campus_dtm.tif'
F_DSM = '/arcgis/home/pycrown4/pycrown/site_data/campus_site/raster/campus_dsm.tif'
#F_LAS = '/arcgis/home/pycrown4/pycrown/site_data/spr_site_1/laz/057847.laz'

In [9]:
#out = '/arcgis/home/pycrown4/pycrown/site_data/spr_site_1/results_SPR/'
#PC = PyCrown(F_CHM, F_DTM, F_DSM, outpath=out)

In [14]:
range(len(selected)-32)

range(0, 5)

In [ ]:
import fiona
from fiona.crs import from_epsg
import shapely
from shapely.geometry import mapping, Point, Polygon
import time

start = time.time()

PC = PyCrown(F_CHM, None, None, None, outpath=out)



#TIF files for each file in in raster_test if .TIF prosses/ if CHM and a TIF. keep Number for nameing

In [7]:
# This works seamlessly within original tiles
# Currently, shapefiles written without prj- fiona crs bug/dependency conflict

import fiona
from fiona.crs import from_epsg
import shapely
from shapely.geometry import mapping, Point, Polygon
import time

start = time.time()

i=0
tile_ext = [None] * len(selected)
selected_id = [None] * len(selected)

F_LAS = None

PC = PyCrown(F_CHM, None, None, None, outpath=out)
dtm_original = PC.dtm
dsm_original = PC.dsm
chm_original=PC.chm0
ul_lon_orig=PC.ul_lon
ul_lat_orig=PC.ul_lat
in_crs = PC.srs

for i in range(len(selected)-35):
    #tile_ext[i] = str(aoi_tiles['SHAPE'][i]['rings'][0][0][0])+','+str(aoi_tiles['SHAPE'][i]['rings'][0][2][0])+','+str(aoi_tiles['SHAPE'][i]['rings'][0][0][1])+','+str(aoi_tiles['SHAPE'][i]['rings'][0][1][1])
    selected_id[i] = selected[i]
    print(selected_id[i])
    xmin = aoi_tiles['SHAPE'][i]['rings'][0][0][0]
    xmax = aoi_tiles['SHAPE'][i]['rings'][0][2][0]
    ymin = aoi_tiles['SHAPE'][i]['rings'][0][0][1]
    ymax = aoi_tiles['SHAPE'][i]['rings'][0][1][1]
    ext = (xmin,xmax,ymin,ymax)
    print(ext)

    #os.mkdir('/arcgis/home/pycrown4/pycrown/site_data/spr_site_1/results4b/%s' % (selected_id[i]))
    #out = '/arcgis/home/pycrown4/pycrown/site_data/spr_site_1/results_SPR/'
    print(out)
    
    PC.chm = None
    PC.chm0 = chm_original
    PC.dtm = dtm_original
    PC.dsm = dsm_original
    
    PC.ul_lon = ul_lon_orig
    PC.ul_lat = ul_lat_orig
    
    PC.trees = pd.DataFrame(columns=[
    'top_height', 'top_elevation',
    'top_cor_height', 'top_cor_elevation',
    'crown_poly_raster', 'crown_poly_smooth',
    'top_cor', 'top', 'tt_corrected'
    ])

    PC.trees = PC.trees.astype(dtype={
    'top_height': 'float',
    'top_elevation': 'float',
    'top_cor_height': 'float',
    'top_cor_elevation': 'float',
    'crown_poly_raster': 'object',
    'crown_poly_smooth': 'object',
    'top_cor': 'object',
    'top': 'object',
    'tt_corrected': 'int'
    })
    
    lon_min, lon_max, lat_min, lat_max = ext
    
    bf = 99
    lon_min_bf = xmin - bf
    lon_max_bf = xmax + bf
    lat_min_bf = ymin - bf
    lat_max_bf = ymax + bf
    
    ext_bf= (lon_min_bf, lon_max_bf, lat_min_bf, lat_max_bf)
    
    #print(ext)
    print(ext_bf)
        
    PC.clip_data_to_bbox(ext_bf)
    
    print(PC.ul_lon, PC.ul_lat)
    
    PC.filter_chm(3,ws_in_pixels=True, circular=False)
        
    try:
        PC.tree_detection(PC.chm, ws=3, ws_in_pixels=True, hmin=6)

        print(f"Number of trees detected: {len(PC.trees)}")  

        
        PC.crown_delineation(algorithm='dalponteCIRC_numba', th_tree=6, th_seed=0.45, th_crown=0.55, max_crown=30)

        
        PC.clip_trees_to_bbox(bbox=ext)
        #Lets skip the 'correct tree top height' step, since it takes forever
        #PC.correct_tree_tops()
        PC.get_tree_height_elevation(loc='top')

        #and here
        #PC.get_tree_height_elevation(loc='top_cor')
        PC.screen_small_trees(hmin=6, loc='top')

        PC.crowns_to_polys_raster()

        #and here, this takes forever also
        #PC.crowns_to_polys_smooth(store_las=False)

        #skip
        #PC.quality_control()
        #PC.export_raster(PC.chm, PC.outpath / 'chm.tif', 'CHM')

        #Tree Tops Export
        out_trees = out+'tree_tops_'+selected_id[i]+'.shp'
        tree_schema = {'geometry': '3D Point',
                'properties': {'DN': 'int', 'TH': 'float'}}

        with fiona.collection(str(out_trees), 'w', 'ESRI Shapefile', tree_schema
            ) as output:
                for tidx in range(len(PC.trees['top'])):
                    feat = {}
                    tree_top = PC.trees['top'].iloc[tidx]
                    feat['geometry'] = mapping(PC.trees['top'].iloc[tidx])
                    feat['properties'] = {'DN': tidx,'TH': float(PC.trees.iloc[tidx].top_height)
                    }
                    output.write(feat)

            #skipping this
            #PC.export_tree_locations(loc='top_cor')

        outfile = out+'tree_crown_'+selected_id[i]+'.shp' 
        crown_schema = {'geometry': 'Polygon',
                  'properties': {'DN': 'int', 'TTH': 'float'}}
        
        with fiona.collection(str(outfile), 'w', 'ESRI Shapefile', crown_schema
            ) as output:
                for tidx in range(len(PC.trees['crown_poly_raster'])):
                    feat = {}
                    tree = PC.trees['crown_poly_raster'].iloc[tidx]
                    feat['geometry'] = mapping(PC.trees['crown_poly_raster'].iloc[tidx])
                    feat['properties'] = {'DN': tidx,'TTH': float(PC.trees.iloc[tidx].top_height)
                    }
                    output.write(feat)

    except:
    
        pass
    
    endtile = time.time()
    
    print(endtile - start)   



NameError: name 'selected' is not defined

#### PyCrown Parameters
    th_tree :   float
                Threshold below which a pixel cannot be a tree. Default 2
    th_seed :   float
                Growing threshold 1. A pixel is added to a region if its height
                is greater than the tree height multiplied by this value. It
                should be between 0 and 1. Default 0.45
    th_crown :  float
                Growing threshold 2. A pixel is added to a region if its height
                is greater than the current mean height of the region
                multiplied by this value. It should be between 0 and 1.
                Default 0.55.
    max_crown : float
                Maximum value of the crown diameter of a detected tree (in
                pixels). Default 10

In [20]:
ext

(6058950.55596525, 6060720.56943591, 1849756.12732558, 1851406.73192965)

In [13]:
# This works with one extent aoi_bbox
# Currently, shapefiles written without prj- fiona crs bug/dependency conflict

import fiona
from fiona.crs import from_epsg
import shapely
from shapely.geometry import mapping, Point, Polygon
import time
start = time.time()
F_LAS = None
PC = PyCrown(F_CHM, F_DTM, F_DSM, F_LAS, outpath=out)
ext = aoi_bbox
print(ext)
print(out)

PC.clip_data_to_bbox(ext)

PC.filter_chm(3,ws_in_pixels=True, circular=False)

d_ws = 3

PC.tree_detection(PC.chm, ws=d_ws, ws_in_pixels=True, hmin=9)
print(f"Number of trees detected: {len(PC.trees)}")  

#def 6, 0.7, 0.55, 32

t_ht =6
t_sd =0.45
t_cr =0.55
t_mx =50

PC.crown_delineation(algorithm='dalponteCIRC_numba', th_tree=t_ht, th_seed=t_sd, th_crown=t_cr, max_crown=t_mx)


PC.clip_trees_to_bbox(bbox=ext)
#Lets skip the 'correct tree top height' step, since it takes forever
#PC.correct_tree_tops()
PC.get_tree_height_elevation(loc='top')

#and here
#PC.get_tree_height_elevation(loc='top_cor')
PC.screen_small_trees(hmin=6, loc='top')

PC.crowns_to_polys_raster()

#and here, this takes forever also
#PC.crowns_to_polys_smooth(store_las=False)

#skip
#PC.quality_control()
#PC.export_raster(PC.chm, PC.outpath / 'chm.tif', 'CHM')

#Tree Tops Export

s_ws = str(d_ws).replace('.', '')
s_sd = str(t_sd).replace('.', '')
s_cr = str(t_cr).replace('.', '')
s_mx = str(t_mx)

out_trees = out+'tree_tops_'+s_ws+'ws_'+s_sd+'sd_'+s_cr+'cr'+s_mx+'mx'+'.shp'
tree_schema = {'geometry': '3D Point',
        'properties': {'DN': 'int', 'TH': 'float'}}

with fiona.collection(str(out_trees), 'w', 'ESRI Shapefile', tree_schema
    ) as output:
        for tidx in range(len(PC.trees['top'])):
            feat = {}
            tree_top = PC.trees['top'].iloc[tidx]
            feat['geometry'] = mapping(PC.trees['top'].iloc[tidx])
            feat['properties'] = {'DN': tidx,'TH': float(PC.trees.iloc[tidx].top_height)
            }
            output.write(feat)

    #skipping this
    #PC.export_tree_locations(loc='top_cor')

        
outfile = out+'tree_crown_'+s_ws+'ws_'+s_sd+'sd_'+s_cr+'cr_'+s_mx+'mx'+'.shp' 
crown_schema = {'geometry': 'Polygon',
          'properties': {'DN': 'int', 'TTH': 'float'}}

with fiona.collection(str(outfile), 'w', 'ESRI Shapefile', crown_schema
    ) as output:
        for tidx in range(len(PC.trees['crown_poly_raster'])):
            feat = {}
            tree = PC.trees['crown_poly_raster'].iloc[tidx]
            feat['geometry'] = mapping(PC.trees['crown_poly_raster'].iloc[tidx])
            feat['properties'] = {'DN': tidx,'TTH': float(PC.trees.iloc[tidx].top_height)
            }
            output.write(feat)



endtile = time.time()

print(endtile - start)   



(5763402.2782367, 5770203.72295616, 2304625.17225101, 2310503.98621985)
/arcgis/home/pycrown4/pycrown/site_data/campus_site/results_2/
Number of trees detected: 11010
Tree crowns delineation: 0.210s
57.22370648384094


In [27]:
outfile

'/arcgis/home/pycrown4/pycrown/site_data/campus_site/results_1tree_crown_3ws_045sd_055cr_50mx.shp'

In [ ]:
#Create Spatially Enabled Data Frame

from arcgis.features import SpatialDataFrame, Feature, FeatureSet
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import pandas as pd
from arcgis import geometry

# read shp into Spatially Enabled Data Frame
crowns_sdf = pd.DataFrame.spatial.from_featureclass(outfile)
crowns_sdf.head()

In [79]:
#crowns_sdf.spatial.set_geometry('SHAPE',sr=6420)

In [102]:
# Define CRS for all features
for i in range(len(crowns_sdf)):
    f=crowns_sdf.loc[i]
    f.SHAPE['spatialReference'] = {'wkid': 6420}
    print(f)
    

FID                                                      0
DN                                                       0
TTH                                                 50.242
SHAPE    {'rings': [[[6060042.55596525, 1851406.7319296...
Name: 0, dtype: object
FID                                                      1
DN                                                       1
TTH                                                80.7448
SHAPE    {'rings': [[[6059157.55596525, 1851406.7319296...
Name: 1, dtype: object
FID                                                      2
DN                                                       2
TTH                                                130.101
SHAPE    {'rings': [[[6059487.55596525, 1851406.7319296...
Name: 2, dtype: object
FID                                                      3
DN                                                       3
TTH                                                107.524
SHAPE    {'rings': [[[6059829.55596525, 185140

In [113]:
#Create Map 
m1 = GIS().map()
m1.basemap = 'imagery'
m1.center = {'x':-13604816,'y':4446238,"spatialReference" : {"wkid" : 3857}}
m1.zoom = 17
from ipywidgets import *
m1.layout=Layout(flex='1 1', height='800px')
m1

MapView(layout=Layout(flex='1 1', height='800px'))

In [127]:
#Get extent of crowns
mapx_min = crowns_sdf.spatial.full_extent[0]
mapx_max = crowns_sdf.spatial.full_extent[2]
mapy_min = crowns_sdf.spatial.full_extent[1]
mapy_max = crowns_sdf.spatial.full_extent[3]

map_ext = {'xmin': mapx_min, 'ymin': mapy_min, 'xmax': mapx_max, 'ymax': mapy_max, 'spatialReference' : {'wkid' : 6420}}
map_ext

{'xmin': 6058950.55596525,
 'ymin': 1849756.73192965,
 'xmax': 6060720.55596525,
 'ymax': 1851406.73192965,
 'spatialReference': {'wkid': 6420}}

In [129]:
#Plot crowns on Map, zoom to extent
from arcgis.mapping import display_colormaps
from arcgis.mapping import show_styles

crowns_sdf.spatial.plot(map_widget=m1,
                           renderer_type='s',
                           cmap = 'viridis',  # use a red to purple color map
                           symbol_type='simple',
                           symbol_style='s',
                           outline_style='s',
                           outline_color=[255,255,255,255],
                           line_width=1.0,
                           alpha = 0.15)

m1.extent = map_ext

In [128]:
#Remove map layers
m1.remove_layers()

True

Haven't update any steps beyond this point

### Notes:
 - currently, 268s (4.45 min) for 37 tiles, or, 7s per tile (0.58km2). 
 - Equates to 5 km2 per min in forested env
 - 169,000 trees, exported as points and crown polygons

In [40]:
from arcgis.features import SpatialDataFrame, Feature, FeatureSet
import pandas as pd
from arcgis import geometry

#shape = {
# "rings": [[[-90.0, 30.0], [-91.0, 30.0], [-90.0, 31.0]]],
# "spatialreference" : {"wkid" : 4326}}

polygonFeature = Polygon(shape)

polygon_new = Feature(geometry=polygonFeature, attributes={"title": "Polygon test",
                                            "description": "This is a test",})
#Create a featureset to access spatial dataframe
polygon_fset = FeatureSet(features = [polygon_new], 
                        geometry_type="Polygon", 
                        spatial_reference={'latestWkid': 4326, 'wkid': 102100})
polygon_fset.sdf‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍

(4620, 9)


### ToDo:
   - Visualize results as web map?
   - Write or append to file via pygeos?
   - set output crs
   - and or, merge shps into top pts, and polygon layers
   - data mgmt\cleanup steps (remove small sliver polygons, smooth polygons?, etc)
   - publish merged layer to AGOL
   - add timing 
   - add error messages
   - actual parameter tuning
   - accuracy assessment
   
   Long term:
   - Create overlapping raster tiles for veryy larg extent input
   - LAZ to DEM, DTM, DSM, CHM
   - LAZ Vegetation classification

In [49]:
# https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.to_file.html#geopandas.GeoDataFrame.to_file
# option to append to file

from shapely import wkt
import geopandas

#PC.trees['top'] = geopandas.GeoSeries.from_wkt(PC.trees['top'])

gdf = geopandas.GeoDataFrame(PC.trees, geometry='top', schema='tree_schema')



TypeError: __init__() got an unexpected keyword argument 'schema'

In [59]:
df_crown = PC.trees[['crown_poly_raster','top_height']]

In [60]:
type(df_crown)

pandas.core.frame.DataFrame

In [77]:
gdf_crown = geopandas.GeoDataFrame(df_crown, geometry='crown_poly_raster')

In [34]:
!pip install pygeos

     |████████████████████████████████| 2.1 MB 28.1 MB/s eta 0:00:01


In [ ]:
conda install -c conda-forge pygeos

In [ ]:
conda list

In [81]:
geopandas.options.use_pygeos = False
print(geopandas.options.use_pygeos)

False


In [82]:
#import pygeos
start_time = time.time() 
gdf_crown.to_file('crown_poly_reg.shp')
print("It takes %s seconds" % (time.time() - start_time)+" without pygoes")

It takes 0.12926101684570312 seconds using pygoes


In [47]:
geopandas.options.use_pygeos = False
print(geopandas.options.use_pygeos)

False


In [43]:
start_time = time.time() 
gdf = geopandas.GeoDataFrame(PC.trees, geometry='top')
print("It takes %s seconds" % (time.time() - start_time)+" without pygoes")

It takes 0.0006866455078125 seconds without pygoes


In [22]:
#os.mkdir('/arcgis/home/pycrown4/pycrown/site_data/spr_site_1/results_SPR_12g/gdf/')
#os.chdir('/arcgis/home/pycrown4/pycrown/site_data/spr_site_1/results_SPR_12g/gdf/')

In [48]:
import pygeos
gdf.to_file('dataframe.shp')

ValueError: Invalid field type <class 'shapely.geometry.polygon.Polygon'>

In [34]:
import os
# return all files as a list
in_files = []
for file in os.listdir(out):
     # check the files which are end with specific extension
    if file.endswith(".shp"):
        # print path name of selected files
        #print(os.path.join(file[:-4]))
        in_f = os.path.join(file)
        in_files.append(in_f)
print(in_files)

['tree_tops_060845.shp', 'tree_tops_067845.shp', 'tree_tops_065840.shp', 'tree_tops_062850.shp', 'tree_tops_070840.shp', 'tree_tops_055840.shp', 'tree_tops_067837.shp', 'tree_tops_060837.shp', 'tree_tops_057845.shp', 'tree_tops_062842.shp', 'tree_tops_055847.shp', 'tree_tops_065845.shp', 'tree_tops_060850.shp', 'tree_tops_067850.shp', 'tree_tops_057837.shp', 'tree_tops_055845.shp', 'tree_tops_060842.shp', 'tree_tops_067842.shp', 'tree_tops_065837.shp', 'tree_tops_065850.shp', 'tree_tops_057842.shp', 'tree_tops_055850.shp', 'tree_tops_062840.shp', 'tree_tops_060847.shp', 'tree_tops_057850.shp', 'tree_tops_065842.shp', 'tree_tops_070837.shp', 'tree_tops_062847.shp', 'tree_tops_060840.shp', 'tree_tops_067840.shp', 'tree_tops_057847.shp', 'tree_tops_062845.shp', 'tree_tops_065847.shp', 'tree_tops_067847.shp', 'tree_tops_057840.shp', 'tree_tops_062837.shp', 'tree_tops_055842.shp']


In [35]:
# Arcpy merge tree top shapefiles into one
# This took a long time

import arcpy
from arcpy import env

env.workspace = out
tree_tops_merge = 'tree_tops_merged.shp'

with arcpy.EnvManager(outputCoordinateSystem='PROJCS["NAD_1983_2011_StatePlane_California_III_FIPS_0403_Ft_US",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",6561666.666666666],PARAMETER["False_Northing",1640416.666666667],PARAMETER["Central_Meridian",-120.5],PARAMETER["Standard_Parallel_1",37.06666666666667],PARAMETER["Standard_Parallel_2",38.43333333333333],PARAMETER["Latitude_Of_Origin",36.5],UNIT["Foot_US",0.3048006096012192]]'):
    arcpy.management.Merge(in_files, tree_tops_merge, "ADD_SOURCE_INFO")

In [ ]:
for i in range(len(selected)):
    crowns = gpd.read_file('/arcgis/home/pycrown4/pycrown/site_data/spr_site_1/results/tree_crown_poly_raster%s.shp'  % (selected['tile_id'][i]))
    print (crowns)
    %matplotlib inline
    crowns.plot(column='DN', cmap=None)
    trees = gpd.read_file('/arcgis/home/pycrown4/pycrown/site_data/spr_site_1/results')
    #print (trees)
    %matplotlib inline
    trees.plot(column='DN', cmap=None)
    

In [ ]:
import zipfile
for i in range(len(selected)):
    os.chdir('/arcgis/home/pycrown4/pycrown/site_data/spr_site_1/results%s' % (selected['tile_id'][i]))
    list_files = ['tree_crown_poly_raster.cpg', 'tree_crown_poly_raster.dbf', 'tree_crown_poly_raster.prj', 'tree_crown_poly_raster.shp', 'tree_crown_poly_raster.shx']
    with zipfile.ZipFile('shape_test1_%s.zip' % (selected['tile_id'][i]), 'w') as zip_file:        

    for file in list_files:
        zip_file.write(file)

        
    zip_file.close
    
    
    

In [ ]:
from IPython.display import display
from arcgis.gis import GIS
import os
gis = GIS('Home')

In [ ]:
for i in range(len(selected)):
    data = "/arcgis/home/pycrown4/pycrown/site_data/spr_site_1/results/shape_test1_%s.zip"  % (selected['tile_id'][i])
    shpfile = gis.content.add({}, data)
    published_service = shpfile.publish()
    display(published_service)

In [ ]:
# From J.Ogden
#TO DO: figure out a way to delete or change the name atomatically (Date and time in file name)
#TO DO: merge tiles befor publishing 
#TO DO: publish and merge tree top data (will be seperate layers)

for i in range(len(selected)):
    selected_id[i] = selected[i]
   
    data = "/arcgis/home/pycrown4/pycrown/site_data/spr_site_1/tile/results_test_%s/shape_test4_%s.zip"  % (selected_id[i],selected_id[i])
    try:
        shpfile = gis.content.add({}, data)
        published_service = shpfile.publish()
        display(published_service)
    except:
        pass

In [25]:
import gdal
import osr
import fiona
from fiona.crs import from_epsg

F_CHM = '/arcgis/home/pycrown4/pycrown/site_data/spr_site_1/raster/SCC_2020_CHM.tif'
PC.chm_file = F_CHM

try:
    chm_gdal = gdal.Open(str(PC.chm_file), gdal.GA_ReadOnly)
except RuntimeError as e:
    raise IOError(e)
proj = osr.SpatialReference(wkt=chm_gdal.GetProjection())
PC.epsg = int(proj.GetAttrValue('AUTHORITY', 1))
PC.srs = from_epsg(PC.epsg)
PC.geotransform = chm_gdal.GetGeoTransform()
PC.resolution = abs(PC.geotransform[-1])
PC.ul_lon = chm_gdal.GetGeoTransform()[0]
PC.ul_lat = chm_gdal.GetGeoTransform()[3]
PC.chm0 = chm_gdal.GetRasterBand(1).ReadAsArray()
chm_gdal = None


In [ ]:
#This works as a large block
import fiona
import shapely
from shapely.geometry import mapping, Point, Polygon
import time

start = time.time()

aoi_bbox

xmin = aoi_bbox[0]
xmax = aoi_bbox[1]
ymin = aoi_bbox[2]
ymax = aoi_bbox[3]
ext = (xmin,xmax,ymin,ymax)
#print(ext)

#os.mkdir('/arcgis/home/pycrown4/pycrown/site_data/spr_site_1/results4b/%s' % (selected_id[i]))
out = '/arcgis/home/pycrown4/pycrown/site_data/spr_site_1/results_SPR_lg/'
print(out)

F_LAS = None

lon_min, lon_max, lat_min, lat_max = ext

bf = 99
lon_min_bf = xmin - bf
lon_max_bf = xmax + bf
lat_min_bf = ymin - bf
lat_max_bf = ymax + bf

ext_bf= (lon_min_bf, lon_max_bf, lat_min_bf, lat_max_bf)

print(ext)
print(ext_bf)
PC = PyCrown(F_CHM, F_DTM, F_DSM, F_LAS, outpath=out)

PC.clip_data_to_bbox(ext_bf)


PC.filter_chm(3,ws_in_pixels=True, circular=False)

try:
    PC.tree_detection(PC.chm, ws=3, ws_in_pixels=True, hmin=6)

    print(f"Number of trees detected: {len(PC.trees)}")  


    PC.crown_delineation(algorithm='dalponteCIRC_numba', th_tree=6, th_seed=0.45, th_crown=0.55, max_crown=30)
    PC.clip_trees_to_bbox(bbox=ext)
    #Lets skip the 'correct tree top height' step, since it takes forever
    #PC.correct_tree_tops()
    PC.get_tree_height_elevation(loc='top')

    #and here
    #PC.get_tree_height_elevation(loc='top_cor')
    PC.screen_small_trees(hmin=6, loc='top')

    PC.crowns_to_polys_raster()

    #and here, this takes forever also
    #PC.crowns_to_polys_smooth(store_las=False)

    #skip
    #PC.quality_control()
    #PC.export_raster(PC.chm, PC.outpath / 'chm.tif', 'CHM')

    #Tree Tops Export
    out_trees = out+'tree_tops.shp'
    tree_schema = {'geometry': '3D Point',
            'properties': {'DN': 'int', 'TH': 'float'}}

    with fiona.collection(
        str(out_trees), 'w', 'ESRI Shapefile', tree_schema
        ) as output:
            for tidx in range(len(PC.trees['top'])):
                feat = {}
                tree_top = PC.trees['top'].iloc[tidx]
                feat['geometry'] = mapping(PC.trees['top'].iloc[tidx])
                feat['properties'] = {'DN': tidx,'TH': float(PC.trees.iloc[tidx].top_height)
                }
                output.write(feat)

    #skipping this
    #PC.export_tree_locations(loc='top_cor')

    outfile = out+'tree_crown.shp'
    schema = {'geometry': 'Polygon',
          'properties': {'DN': 'int', 'TTH': 'float'}}
    with fiona.collection(
        str(outfile), 'w', 'ESRI Shapefile', schema
        ) as output:
            for tidx in range(len(PC.trees['crown_poly_raster'])):
                feat = {}
                tree = PC.trees['crown_poly_raster'].iloc[tidx]
                feat['geometry'] = mapping(PC.trees['crown_poly_raster'].iloc[tidx])
                feat['properties'] = {'DN': tidx,'TTH': float(PC.trees.iloc[tidx].top_height)
                }
                output.write(feat)
    end = time.time()
    print(end - start)   
except:
    end = time.time()
    print(end - start)
    pass


/arcgis/home/pycrown4/pycrown/site_data/spr_site_1/results_SPR_lg/
(6057499.99261454, 6070000.00280755, 1840000.00843127, 1852500.00848761)
(6057400.99261454, 6070099.00280755, 1839901.00843127, 1852599.00848761)
Number of trees detected: 117261
Tree crowns delineation: 1.823s


In [12]:
out_trees = out+'tree_tops.shp'
tree_schema = {'geometry': '3D Point',
        'properties': {'DN': 'int', 'TH': 'float'}}

with fiona.collection(
    str(out_trees), 'w', 'ESRI Shapefile', tree_schema
    ) as output:
        for tidx in range(len(PC.trees['top'])):
            feat = {}
            tree_top = PC.trees['top'].iloc[tidx]
            feat['geometry'] = mapping(PC.trees['top'].iloc[tidx])
            feat['properties'] = {'DN': tidx,'TH': float(PC.trees.iloc[tidx].top_height)
            }
            output.write(feat)

#skipping this
#PC.export_tree_locations(loc='top_cor')

outfile = out+'tree_crown.shp'
schema = {'geometry': 'Polygon',
      'properties': {'DN': 'int', 'TTH': 'float'}}
with fiona.collection(
    str(outfile), 'w', 'ESRI Shapefile', schema
    ) as output:
        for tidx in range(len(PC.trees['crown_poly_raster'])):
            feat = {}
            tree = PC.trees['crown_poly_raster'].iloc[tidx]
            feat['geometry'] = mapping(PC.trees['crown_poly_raster'].iloc[tidx])
            feat['properties'] = {'DN': tidx,'TTH': float(PC.trees.iloc[tidx].top_height)
            }
            output.write(feat)

In [20]:
# Troubleshooting writing dataframe to shp
import time
import gdal
import osr
import pyproj

from shapely.geometry import mapping, Point, Polygon
import fiona

from fiona.crs import from_epsg


outfile = out+'tree_crown_'+selected_id[i]+'.shp' 
crown_schema = {'geometry': 'Polygon', 'properties': {'DN': 'int', 'TTH': 'float'}}

with fiona.collection(str(outfile), 'w', driver='ESRI Shapefile', schema=crown_schema) as output:
    for tidx in range(len(PC.trees['crown_poly_raster'])):
        feat = {}
        tree = PC.trees['crown_poly_raster'].iloc[tidx]
        feat['geometry'] = mapping(PC.trees['crown_poly_raster'].iloc[tidx])
        feat['properties'] = {'DN': tidx,'TTH': float(PC.trees.iloc[tidx].top_height)}
        output.write(feat)
